In [1]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sn
import cv2

import glob

import numpy as np

In [2]:
train_dir = '../input/chest-xray-pneumonia/chest_xray/train'
test_dir = '../input/chest-xray-pneumonia/chest_xray/test'
val_dir = '../input/chest-xray-pneumonia/chest_xray/val'

In [3]:
pneumonia_train_images = glob.glob(train_dir+"/PNEUMONIA/*.jpeg")
normal_train_images = glob.glob(train_dir+"/NORMAL/*.jpeg")

In [4]:
pneumonia_val_images = glob.glob(val_dir+"/PNEUMONIA/*.jpeg")
normal_val_images = glob.glob(val_dir+"/NORMAL/*.jpeg")

In [5]:
pneumonia_test_images = glob.glob(test_dir+"/PNEUMONIA/*.jpeg")
normal_test_images = glob.glob(test_dir+"/NORMAL/*.jpeg")

# Preprocessing and X-ray image Visualization

In [41]:
fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(pneumonia_train_images[i])
    img = cv2.resize(img, (512,512))
    ax.imshow(img)
    ax.set_title("Pneumonia")
    
plt.show()

fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(normal_train_images[i])
    img = cv2.resize(img, (220,220))
    ax.imshow(img)
    ax.set_title("Normal")
#fig.tight_layout()    
plt.show()

In [42]:
fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(pneumonia_train_images[i])
    img = cv2.resize(img, (512,512))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.Canny(img, 80, 100)
    ax.imshow(img)
    ax.set_title("Pneumonia")
fig.tight_layout()
plt.show()

fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(normal_train_images[i])
    img = cv2.resize(img, (512,512))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.Canny(img, 80, 100)
    
    ax.imshow(img)
    ax.set_title("Normal")
fig.tight_layout()    
plt.show()

## Heatmap Visualization

In [45]:
fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(pneumonia_train_images[i])
    img = cv2.resize(img, (512,512))
    img = cv2.applyColorMap(img, cv2.COLORMAP_JET)
    #img = cv2.Canny(img, 80, 100)
    #print(img.shape)
    ax.imshow(img)
    ax.set_title("Pneumonia")
fig.tight_layout()
plt.show()

fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(normal_train_images[i])
    img = cv2.resize(img, (512,512))
    #print(img.shape)
    img = cv2.applyColorMap(img, cv2.COLORMAP_JET)
    #img = cv2.Canny(img, 80, 100)
    ax.imshow(img)
    ax.set_title("Normal")
fig.tight_layout()    
plt.show()

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   zoom_range = 0.2)
val_datagen = ImageDataGenerator(rescale = 1./255.,)
test_datagen = ImageDataGenerator(rescale = 1./255.,)


train_generator = train_datagen.flow_from_directory(train_dir, batch_size=20, class_mode='binary', target_size = (220, 220))
validation_generator = val_datagen.flow_from_directory(val_dir, batch_size=20, class_mode = 'binary', target_size=(220, 220))
test_generator = test_datagen.flow_from_directory(test_dir,shuffle=False, batch_size=20, class_mode = 'binary', target_size=(220, 220))

In [7]:
import ssl

input_shape = (220, 220, 3)
ssl._create_default_https_context = ssl._create_unverified_context
base_model = tf.keras.applications.ResNet50V2(weights='imagenet', input_shape=input_shape, include_top=False)
#base_model = tf.keras.applications.VGG16(include_top=False,weights="imagenet",input_shape=input_shape)

for layer in base_model.layers:
    layer.trainable = False
    
    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) #Output
model.summary()

In [8]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=["accuracy"])

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)

history = model.fit(train_generator, validation_data=validation_generator, steps_per_epoch = 100, epochs=20, callbacks=callback)

In [17]:
model.save("model1")

In [9]:
accuracy = history.history['accuracy']
val_accuracy  = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [10]:
plt.figure(figsize=(15,10))

plt.subplot(2, 2, 1)
plt.plot(accuracy, label = "Training accuracy")
plt.plot(val_accuracy, label="Validation accuracy")
plt.ylim(0.7, 1)
plt.legend()
plt.title("Training vs validation accuracy")


plt.subplot(2,2,2)
plt.plot(loss, label = "Training loss")
plt.plot(val_loss, label="Validation loss")
plt.ylim(0, 0.5)
plt.legend()
plt.title("Training vs validation loss")

plt.show()

In [12]:
score = model.evaluate(test_generator, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# Evaluating Model Performance Using RGB Images(Heatmap)

In [13]:
def heatmap_convert(train_images, target_label, data_list, target_list):
    for img_name in train_images:
        img = cv2.imread(img_name)
        img = cv2.resize(img, (220,220))
        img1 = cv2.applyColorMap(img, cv2.COLORMAP_JET)
        img2 = img
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.Canny(img, 80, 100)
        img3 = np.zeros_like(img)
        img3[:,:,0] = gray
        img3[:,:,1] = gray
        img3[:,:,2] = gray
        #img3 = cv2.applyColorMap(img, cv2.COLORMAP_OCEAN)
        img = np.concatenate([img3,img2, img1], axis=-1)
        data_list.append(img)
        target_list.append(target_label)
    return data_list, target_list


In [14]:
data = []
target = []
train_data, train_target = heatmap_convert(pneumonia_train_images, 1, data, target)
train_data, train_target = heatmap_convert(normal_train_images, 0, train_data, train_target)
print(len(train_data),len(train_target))

In [15]:
data_list = []
target_list = []
val_data, val_target = heatmap_convert(pneumonia_val_images, 1, data_list, target_list)
val_data, val_target = heatmap_convert(normal_val_images, 0, val_data, val_target)
print(len(val_data),len(val_target))

In [16]:
test_data_list = []
test_target_list = []
test_data, test_target = heatmap_convert(pneumonia_test_images, 1, test_data_list, test_target_list)
test_data, test_target = heatmap_convert(normal_test_images, 0, test_data, test_target)
print(len(test_data),len(test_target))

In [17]:
data = np.array(train_data)
target = np.array(train_target)
print(data.shape,target.shape)

#np.save('data',data)
#np.save('target',target)
    

In [18]:
val_data = np.array(val_data)
val_target = np.array(val_target)
print(val_data.shape,val_target.shape)
#np.save('val_data',val_data)
#np.save('val_target',val_target)

In [19]:
test_data = np.array(test_data)
test_target = np.array(test_target)
print(test_data.shape,test_target.shape)

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,Dropout,BatchNormalization
from keras import regularizers
input_shape = (220, 220, 9)
model = Sequential([
    Conv2D(64,input_shape=input_shape,kernel_size=(3,3),activation='relu',kernel_regularizer=regularizers.l2(l=0.0001),padding = "same",name='conv_1'),
    BatchNormalization(momentum = 0.99),
    MaxPooling2D(pool_size=(2, 2), name='pool_1'),
    Conv2D(64,input_shape=input_shape,kernel_size=(3,3),activation='relu',kernel_regularizer=regularizers.l2(l=0.0001),padding = "same",name='conv_2'),
    MaxPooling2D(pool_size=(2, 2), name='pool_2'),
    Conv2D(64,input_shape=input_shape,kernel_size=(3,3),activation='relu',kernel_regularizer=regularizers.l2(l=0.0001),padding = "same",name='conv_3'),
    MaxPooling2D(pool_size=(2, 2), name='pool_3'),
    Conv2D(128,input_shape=input_shape,kernel_size=(3,3),activation='relu',kernel_regularizer=regularizers.l2(l=0.0001),padding = "same",name='conv_4'),
    MaxPooling2D(pool_size=(2, 2), name='pool_4'),
    Conv2D(128,input_shape=input_shape,kernel_size=(3,3),activation='relu',kernel_regularizer=regularizers.l2(l=0.0001),padding = "same",name='conv_5'),
    MaxPooling2D(pool_size=(2, 2), name='pool_5'),
    Dense(128,activation='relu',name='dense_1',input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
    Dropout(0.5),
    Dense(128,activation='relu',name='dense_2',kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
    Dropout(0.5),
    Dense(128,activation='relu',name='dense_3',kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
    Dropout(0.5),
    BatchNormalization(momentum = 0.99),
    Flatten(name='flatten'),
    Dense(1, activation='sigmoid',name='dense_4')])

# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='binary_crossentrop', optimizer=sgd, metrics=['accuracy'])
    
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [37]:
#model.compile(optimizer="adam", loss='binary_crossentropy', metrics=["accuracy"])

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)
validation_data=(val_data, val_target)

history = model.fit(x = data, y = target, validation_data=validation_data, steps_per_epoch = 100, epochs=20, callbacks=callback)
#history = model.fit(train_generator, validation_data=validation_generator, steps_per_epoch = 100, epochs=20, callbacks=callback)

In [ ]:
model.save("model1")

In [38]:
accuracy = history.history['accuracy']
val_accuracy  = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [39]:
plt.figure(figsize=(15,10))

plt.subplot(2, 2, 1)
plt.plot(accuracy, label = "Training accuracy")
plt.plot(val_accuracy, label="Validation accuracy")
plt.ylim(0.8, 1)
plt.legend()
plt.title("Training vs validation accuracy")


plt.subplot(2,2,2)
plt.plot(loss, label = "Training loss")
plt.plot(val_loss, label="Validation loss")
plt.ylim(0, 0.5)
plt.legend()
plt.title("Training vs validation loss")

plt.show()

In [40]:
score = model.evaluate(test_data,test_target, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')